In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from nltk.corpus import stopwords
from tqdm import tqdm
import lightgbm as ltb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

In [ ]:
def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [ ]:
def accuracy(pred, y):
    return sum([x==y for x,y in zip(pred, y)]) / len(pred)
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

## Would-read baseline: just rank which books are popular and which are not, and return '1' if a book is among the top-ranked

In [ ]:
ratingsTrain = allRatings[:180000]
ratingsValid = allRatings[180000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
len(allRatings)

In [ ]:
bookCount = defaultdict(int)
totalRead = 0

In [ ]:
for user,book,_ in readCSV("train_Interactions.csv.gz"):
  bookCount[book] += 1
  totalRead += 1

In [ ]:
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

calculate most popular books

In [ ]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead/2: break

In [ ]:
totalbook = set()
data_ub = []
booksPerUser = defaultdict(set)

In [ ]:
for u,b,r in ratingsTrain:
    totalbook.add(b)
    booksPerUser[u].add(b)
    
for u,b,r in ratingsValid:
    totalbook.add(b)
    data_ub.append((u,b))
    booksPerUser[u].add(b)

In [ ]:
validation_0 = []
for i in data_ub:
    u = i[0]
    validation_0.append((u, random.sample(totalbook.difference(booksPerUser[u]), 1)))

1 represents read the book

In [ ]:
validation_set = []
for i in data_ub:
    validation_set.append([i[0], i[1], 1])
for j in validation_0:
    validation_set.append([j[0], j[1][0], 0])

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rateduser
for u,b,r in ratingsTrain:
    usersPerItem[b].add(u)
    itemsPerUser[u].add(b)

In [ ]:
book_count = {}
for ic, i in mostPopular:
    book_count[i] = ic
avg_count = numpy.mean([i[0] for i in mostPopular])
def feature(u,b):
    Jaccard_u = []
    for b2 in itemsPerUser[u]:
        if b2 == b:
            continue
        Jaccard_u.append(Jaccard(usersPerItem[b2], usersPerItem[b]))
    if Jaccard_u == []:
        tocompare = 0
    else: tocompare = numpy.max(Jaccard_u)
    Jaccard_u1 = []
    for u2 in usersPerItem[b]:
        if u2 == u:
            continue
        Jaccard_u1.append(Jaccard(itemsPerUser[u2], itemsPerUser[u]))
    if Jaccard_u1 == []:
        tocompare1 = 0
    else: tocompare1 = numpy.max(Jaccard_u1)
    book_pop = avg_count
    if b in book_count:
        book_pop = book_count[b]
    return [1, book_pop, tocompare, tocompare1]

In [ ]:
X = [feature(i[0],i[1]) for i in validation_set]
y = [i[2] for i in validation_set]

In [ ]:
best_acc = 0
best_model = 0
for c in tqdm(range(-10, 10)):
    acc_avg = []
    acc_max_model = 0
    acc_max = 0
    for i in range(0,10):
        model = linear_model.LogisticRegression(C = 10 ** c, fit_intercept=False)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        model.fit(X_train, y_train)
        prediction = model.predict(X_test)
        acc = accuracy(prediction, y_test)
        acc_avg.append(acc)
        if acc > acc_max:
            acc_max = acc
            acc_max_model = model
    acc_avg = numpy.average(numpy.array(acc_avg))
    print(acc_avg, c)
    if acc_avg > best_acc:
        best_model = acc_max_model
        best_acc = acc_avg
print(best_acc, best_model.coef_)

In [ ]:
best_acc

## Category prediction baseline: Just consider some of the most common words from each category

In [ ]:
catDict = {
  "children": 0,
  "comics_graphic": 1,
  "fantasy_paranormal": 2,
  "mystery_thriller_crime": 3,
  "young_adult": 4
}

In [ ]:
data = []

In [ ]:
for d in readGz("train_Category.json.gz"):
    data.append(d)

hw3 + distinctive feature for each genre

In [ ]:
data[0]

In [ ]:
punctuation = set(string.punctuation)
dfs = [defaultdict(int), defaultdict(int), defaultdict(int), defaultdict(int), defaultdict(int)]
for d in data:
    df = dfs[d['genreID']]
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in set(r.split()):
        df[w] += 1
for i in stopwords.words('english'):
    for df in dfs:
        if i in df:
            del df[i]
counts = [sorted([(v,k) for k, v in df.items()], reverse=True) for df in dfs]

In [ ]:
len(counts)

In [ ]:
words_cat = [set()] * 5
for idx in tqdm(range(5)):
    for i in counts[idx][:5000]:
        words_cat[idx].add(i[1])

In [ ]:
all_five_cat_words = words_cat[0]
for i in words_cat:
    all_five_cat_words = all_five_cat_words.union(i)

In [ ]:
wordId = dict(zip(all_five_cat_words, range(len(all_five_cat_words))))
wordSet = set(all_five_cat_words)

In [ ]:
len(wordSet)

In [ ]:
all_five_cat_words

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(all_five_cat_words)

In [ ]:
vectorizer.transform([data[0]['review_text']])

In [ ]:
def feature_vector(datum):
    feat = vectorizer.transform([datum['review_text']]).toarray()
    return feat[0]
X = [feature_vector(d) for d in data]
y = [d['genreID'] for d in data]

In [ ]:
Xtrain = numpy.array(X[:8*len(X)//10])
ytrain = numpy.array(y[:8*len(y)//10])
Xvalid = numpy.array(X[8*len(X)//10:])
yvalid = numpy.array(y[8*len(y)//10:])

lgbm

In [ ]:
mod = ltb.LGBMClassifier(num_leaves = 100, n_estimators = 1000, learning_rate = 0.07, reg_alpha = 1, reg_lambda = 0.1, n_jobs = -1,objective = 'multiclass')
mod.fit(Xtrain, ytrain)
pred = mod.predict(Xvalid)
correct = pred == yvalid
print(sum(correct) / len(correct))

In [ ]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

In [ ]:
test_data = []
for l in readGz('test_Category.json.gz'):
    test_data.append(l)

In [ ]:
for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    
    x = feature_vector(test_data[pos])
    
#     prediction = best_model.predict([x])
    prediction = mod.predict([x])
    print(l[:-1] + "," + str(prediction[0]) + "\n")
    predictions.write(l[:-1] + "," + str(prediction[0]) + "\n")
    pos += 1
predictions.close()

n_estimator: 1000, learning_rate:0.04 0.7395<br>
n_estimator: 1000, learning_rate: 0.07, reg_alpha: 1, reg_lambda = 0.1, acc: 0.7507<br>
:3000 len(feature) = 4867

no stem

In [ ]:
best_model = 0
best_acc = 0
for c in tqdm(range(-2, 4)):
    mod = linear_model.LogisticRegression(C=10 ** c)
    mod.fit(Xtrain, ytrain)
    pred = mod.predict(Xvalid)
    correct = pred == yvalid
    if sum(correct) / len(correct) > best_acc:
        best_model = mod
        best_acc = sum(correct) / len(correct)
    print(c, sum(correct) / len(correct))